In [56]:
# Cache me ousside how bow dah
import requests

# Extract song info from html
from bs4 import BeautifulSoup
billboard_url = "http://www.billboard.com/articles/list/2155531/the-hot-100-all-time-top-songs"
soup = BeautifulSoup((requests.get(billboard_url)).text, "html.parser")
songs = [song.text for song in soup.find_all("h3", {"class": "list-data__title"})]

# Extra title, artist, and year info
import re
# We consider only the first year for a song if more than one: the 100th song's case
titles  = [str(re.sub(r"\xe2\x80\xa6", r"...", re.sub("\"", "", re.match(r"\"[\"\']?.+\"[\"\']?", song).group(0)))) for song in songs]
artists = [str(re.sub("^\"|\r|(- )|(.*\".*\" )","", re.search("\" .*\r", song).group(0)).lstrip()) for song in songs]
years   = [int(re.sub("\)|\(|,", "", re.search("(\(\d{4}\))|(\(\d{4},)", song).group(0))) for song in songs]

import pandas as pd 
top100songs = pd.DataFrame({'Title': titles, 'Artist': artists, 'Year': years})

In [9]:
top100songs.loc[53]

Artist                                           Elton John
Title     Candle in the Wind 1997 / Something About the ...
Year                                                   1997
Name: 53, dtype: object

In [60]:
import requests_cache
requests_cache.install_cache("cache")
from bs4 import BeautifulSoup
import re
import string
import pandas as pd
test = top100songs.loc[9] 
def processString(s, type):
    s = re.sub("['?:!.,;&/]+", " ", s).lower().strip()
    if(type != "Artist"):
        s = re.sub("-", "",s)
    s = re.sub("\s+", "-", s)
    return(s)

def getLyrics(song, newUrl = ''):
    artist = song['Artist']
    title = song['Title']
    #Remove punctuation and change to lower case.
    artist = processString(artist, "Artist")
    title = processString(title," Title")
    lyric_url = "http://www.songlyrics.com/"+artist+"/"+title+"-lyrics/"
    if newUrl != '':
        lyric_url = newUrl
#    print(lyric_url)
    soup = BeautifulSoup((requests.get(lyric_url)).text, "html.parser")
    tmp = soup.find_all('p', {"id": "songLyricsDiv"})
    lyrics = tmp[0].get_text()
    if bool(re.search(pattern= "(We do not have)|(we have no)", string = lyrics)):
        lyrics = ""
        lyric_url = ""
    return({'Url':lyric_url, 'Lyric': lyrics})


In [58]:
getLyrics(top100songs.loc[80])
top100songs.loc[80]

http://www.songlyrics.com/percy-faith/the-theme-from-a-summer-place-lyrics/


Artist                        Percy Faith
Title     The Theme from 'A Summer Place'
Year                                 1960
Name: 80, dtype: object

In [76]:
result = [getLyrics(row) for idx, row in top100songs.iterrows()]
result = pd.DataFrame(result)

In [70]:
#Locate songs that couldn't be found using function
missing =result[result['Url'] == ""].index
top100songs.loc[missing]

,Artist,Title,Year
53,Elton John,Candle in the Wind 1997 / Something About the ...,1997
60,Paul McCartney and Michael Jackson,"Say, Say, Say",1983
80,Percy Faith,The Theme from 'A Summer Place',1960
83,Jewel,You Were Meant for Me / Foolish Games,1997


In [71]:
notFoundUrls = ["http://www.songlyrics.com/elton-john-billy-joel/candle-in-the-wind-lyrics/", "http://www.songlyrics.com/percy-faith/theme-from-a-summer-place-lyrics/","http://www.songlyrics.com/jewel-feat-kelly-clarkson/foolish-games-lyrics/", ""]

In [91]:
result.loc[53,'Lyric':'Url'] = getLyrics(top100songs.loc[53], newUrl = notFoundUrls[1])
result.loc[80,'Lyric':'Url'] = getLyrics(top100songs.loc[80], newUrl = notFoundUrls[1])
result.loc[83,'Lyric':'Url'] = getLyrics(top100songs.loc[83], newUrl = notFoundUrls[1])

In [95]:
song = top100songs.loc[60]
artist = song['Artist']
title = song['Title']
lyric_url = "http://www.metrolyrics.com/say-say-say-lyrics-paul-mccartney.html"
artist = processString(artist, "Artist")
title = processString(title," Title")
soup = BeautifulSoup((requests.get(lyric_url)).text, "html.parser")
tmp = soup.find_all('div', {"id": "lyrics-body-text"})
lyrics = tmp[0].get_text()
result.loc[60,'Lyric': 'Url'] = {'Lyric': lyrics, 'Url': lyric_url}

In [97]:
result

Lyric    \nSay, say, say what you want but don't play g...
Url      http://www.metrolyrics.com/say-say-say-lyrics-...
Name: 60, dtype: object